In [9]:
!pip install simpy

In [10]:
import simpy
import random
import numpy as np

# Simulation parameters
SIM_TIME = 24 * 60  # 24 hours in minutes
NUM_AGENTS = 4
NUM_SHIFTS = 4
CALLS_PER_DAY = 130
AVG_CALL_DURATION = 4  # minutes

# Calculate inter-arrival time
INTER_ARRIVAL_TIME = SIM_TIME / CALLS_PER_DAY

def contact_center(env, num_agents):
    agents = simpy.Resource(env, capacity=num_agents)
    while True:
        yield env.timeout(random.expovariate(1.0 / INTER_ARRIVAL_TIME))
        env.process(handle_call(env, agents))

def handle_call(env, agents):
    arrival_time = env.now
    with agents.request() as request:
        yield request
        wait_time = env.now - arrival_time
        call_duration = random.expovariate(1.0 / AVG_CALL_DURATION)
        yield env.timeout(call_duration)
        handle_time = env.now - arrival_time
        call_data.append((arrival_time, wait_time, call_duration, handle_time))

# Run simulation
env = simpy.Environment()
call_data = []
env.process(contact_center(env, NUM_AGENTS))
env.run(until=SIM_TIME)

# Calculate overall metrics
arrival_times, wait_times, call_durations, handle_times = zip(*call_data)
total_calls = len(call_data)

avg_wait_time = np.mean(wait_times)
avg_call_duration = np.mean(call_durations)
avg_handle_time = np.mean(handle_times)
overall_utilization = sum(call_durations) / (SIM_TIME * NUM_AGENTS) * 100

print(f"Total calls: {total_calls}")
print(f"Average wait time: {avg_wait_time:.2f} minutes")
print(f"Average call duration: {avg_call_duration:.2f} minutes")
print(f"Average handle time: {avg_handle_time:.2f} minutes")
print(f"Overall agent utilization: {overall_utilization:.2f}%")

# Calculate utilization rates per hour
hourly_utilization = [0] * 24
for arrival_time, _, call_duration, _ in call_data:
    hour = int(arrival_time // 60)
    hourly_utilization[hour] += call_duration

for hour, utilization in enumerate(hourly_utilization):
    utilization_rate = (utilization / (60 * NUM_AGENTS)) * 100
    print(f"Hour {hour}: Utilization rate = {utilization_rate:.2f}%")

Total calls: 137
Average wait time: 0.00 minutes
Average call duration: 4.15 minutes
Average handle time: 4.15 minutes
Overall agent utilization: 9.87%
Hour 0: Utilization rate = 14.39%
Hour 1: Utilization rate = 6.21%
Hour 2: Utilization rate = 6.11%
Hour 3: Utilization rate = 11.58%
Hour 4: Utilization rate = 24.56%
Hour 5: Utilization rate = 0.00%
Hour 6: Utilization rate = 7.67%
Hour 7: Utilization rate = 11.58%
Hour 8: Utilization rate = 6.74%
Hour 9: Utilization rate = 4.27%
Hour 10: Utilization rate = 5.05%
Hour 11: Utilization rate = 8.01%
Hour 12: Utilization rate = 11.70%
Hour 13: Utilization rate = 3.92%
Hour 14: Utilization rate = 18.83%
Hour 15: Utilization rate = 12.76%
Hour 16: Utilization rate = 2.28%
Hour 17: Utilization rate = 11.22%
Hour 18: Utilization rate = 14.02%
Hour 19: Utilization rate = 5.31%
Hour 20: Utilization rate = 5.96%
Hour 21: Utilization rate = 18.89%
Hour 22: Utilization rate = 20.45%
Hour 23: Utilization rate = 5.30%
